In [1]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# data preprocession

boston_data = load_boston()
X = boston_data.data
y = boston_data.target.reshape(X.shape[0],1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
x1 = [[1] for i in range(X_train.shape[0])]
x2 = [[1] for i in range(X_test.shape[0])]
X_train = np.concatenate((x1, X_train), axis=1)
X_test = np.concatenate((x2, X_test), axis=1)
X_train.shape, y_train.shape
X_test.shape, y_test.shape

((404, 14), (404, 1))

((102, 14), (102, 1))

In [3]:
# linear regression model

class MyLinearRegression:
    def __init__(self, alpha=0.000005):
        self.alpha = alpha
        self.W = [[1] for _ in range(14)]
        
    def forward(self, X, W):
        '''
        Calculate predicted data _y.
        '''
        _y = X.dot(W)
        return _y

    def backward(self, X, W, y):
        '''
        Calculate gradient.
        '''
        dW = X.T.dot(X.dot(W) - y) / X.shape[0]
        return dW
    
    def calRSS(self, _y, y):
        '''
        Calculate RSS.
        '''
        RSS = np.sum((_y-y) ** 2)
        return RSS
    
    def train(self, X, y):
        '''
        Load and train data.
        '''
        self.loadParameters('noScaleChange.npy')
        for b in range(10001):
            # use new weight retrain the data
            W = self.W
            for i in range(5000):
                # update weight W here
                dW = self.backward(X, W, y)
                W -= self.alpha * dW
            self.updateParameters(W)
            if b%1000==0:
                _y = self.forward(X, W)
                RSS = self.calRSS(_y, y)
                print('Epoch', b+1, 'train_RSS:', RSS)
        self.saveParameters(W)
        return W, RSS
    
    def saveParameters(self, W):
        np.save('attemptation', W)
        
    def loadParameters(self, filename):
        try:
            self.W = np.load(filename)
        except:
            print('No previous parameters, training from the start...')
    
    def updateParameters(self, W):
        self.W = W


In [4]:
# Train data here

lr_model = MyLinearRegression(alpha=0.000005)
W, train_RSS = lr_model.train(X_train, y_train)

Epoch 1 train_RSS: 8780.743045723593
Epoch 1001 train_RSS: 8778.214096573836
Epoch 2001 train_RSS: 8775.855188959304
Epoch 3001 train_RSS: 8773.654889200927
Epoch 4001 train_RSS: 8771.602532524419
Epoch 5001 train_RSS: 8769.688171329255
Epoch 6001 train_RSS: 8767.902526943275
Epoch 7001 train_RSS: 8766.236944627013
Epoch 8001 train_RSS: 8764.683351607753
Epoch 9001 train_RSS: 8763.23421793868
Epoch 10001 train_RSS: 8761.88251999232


In [21]:
# Calculate RSS, TSS, R^2 for the training and test data

train_TSS = np.sum((y_train-np.mean(y_train)) ** 2)
train_R2 = 1 - train_RSS/train_TSS

train_RSS, train_TSS, train_R2

(8818.584680388702, 35096.85514851485, 0.7487357587147843)

In [5]:
# W = np.load('attemptation.npy')
_y = X_test.dot(W)
# _y[:10]

test_TSS = np.sum((y_test-np.mean(y_test)) ** 2)
test_RSS = np.sum((y_test-_y) ** 2)
test_R2 = 1 - test_RSS/test_TSS

test_RSS, test_TSS, test_R2

(2526.9031950426624, 7480.045882352942, 0.6621807894248112)